In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard
from sklearn.utils import shuffle

# Define the model
model = Sequential()
model.add(LSTM(32, activation='relu', input_shape=(None, 1376)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')

In [13]:


# Load data from files in DataSetFeatures and Rating directories
data_dir = 'DataSetFeatures'
label_dir = 'Rating'

data_files = os.listdir(data_dir)
label_files = os.listdir(label_dir)

# Ensure that the files are sorted in the same order
data_files.sort()
label_files.sort()

# Initialize lists to store features and labels
matrixOfVideos = []
popularity_labels = []

for data_file, label_file in zip(data_files, label_files):
    # Load features from data file
    features = np.load(os.path.join(data_dir, data_file))

    # Load label from label file
    label = np.load(os.path.join(label_dir, label_file))
    label = label*10000-290
    matrixOfVideos.append(features)
    popularity_labels.append(label)

# Pad sequences to ensure they have the same length
matrixOfVideos_padded = pad_sequences(matrixOfVideos, dtype='float32', padding='post', truncating='post')

popularity_labels = np.array(popularity_labels)


shuffled_matrixOfVideos_padded, shuffled_popularity_labels = shuffle(matrixOfVideos_padded, popularity_labels, random_state=42)



# Train the model
# model.fit(matrixOfVideos_padded, popularity_labels, epochs=1000, batch_size=30, validation_split=0.0, callbacks=[TensorBoard()])
# an use the shuffled data for training
model.fit(shuffled_matrixOfVideos_padded, shuffled_popularity_labels, epochs=1000, batch_size=30, validation_split=0.0, callbacks=[TensorBoard()])

# Save the model
model.save('modelShuffleData.h5')

# Print model summary
model.summary()


Epoch 1/1000
4/4 [==============================] - 1s 52ms/step - loss: 33.9512
Epoch 2/1000
4/4 [==============================] - 0s 52ms/step - loss: 33.4818
Epoch 3/1000
4/4 [==============================] - 0s 48ms/step - loss: 33.3919
Epoch 4/1000
4/4 [==============================] - 0s 45ms/step - loss: 32.9583
Epoch 5/1000
4/4 [==============================] - 0s 48ms/step - loss: 32.7269
Epoch 6/1000
4/4 [==============================] - 0s 54ms/step - loss: 32.7335
Epoch 7/1000
4/4 [==============================] - 0s 61ms/step - loss: 32.4725
Epoch 8/1000
4/4 [==============================] - 0s 47ms/step - loss: 30.7481
Epoch 9/1000
4/4 [==============================] - 0s 48ms/step - loss: 31.1350
Epoch 10/1000
4/4 [==============================] - 0s 44ms/step - loss: 29.2824
Epoch 11/1000
4/4 [==============================] - 0s 46ms/step - loss: 30.7800
Epoch 12/1000
4/4 [==============================] - 0s 47ms/step - loss: 26.5345
Epoch 13/1000
4/4 [======

In [9]:
# len(matrixOfVideos_padded)
# len(matrixOfVideos)
popularity_labels*10000-290

array([[ 5.31477208],
       [ 5.31477208],
       [ 5.31477208],
       [ 5.31477208],
       [ 5.31477208],
       [ 5.31477208],
       [ 5.31477208],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 0.90705985],
       [ 8.84968306],
       [ 8.84968306],
       [ 8.84968306],
       [ 8.84968306],
       [ 8.84968306],
       [ 8.84968306],
       [ 8.84968306],
       [ 8.84968306],
       [ 8.84968306],
       [ 8.84968306],
       [ 8.84968306],
       [ 8.84968306],
       [ 8.84968306],
       [ 1.01491451],
       [ 1.01491451],
       [ 1.01491451],
       [ 1.01491451],
       [ 1.01491451],
       [ 1.01491451],
       [ 1.01491451],
       [ 1.01491451],
       [ 1.01491451],
       [ 1.01491451],
       [ 1.01491451],
       [ 1

In [5]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

# Test data directories
test_data_dir = 'TestDataSetFeatures'
test_label_dir = 'TestRating'

test_data_files = os.listdir(test_data_dir)
test_label_files = os.listdir(test_label_dir)

# Ensure files are sorted in the same order
test_data_files.sort()
test_label_files.sort()

# Initialize lists to store test features and labels
test_matrixOfVideos = []
test_popularity_labels = []

for test_data_file, test_label_file in zip(test_data_files, test_label_files):
    # Load features from test data file
    test_features = np.load(os.path.join(test_data_dir, test_data_file))

    # Load label from test label file
    test_label = np.load(os.path.join(test_label_dir, test_label_file))
    test_label = test_label*10000-290
    test_matrixOfVideos.append(test_features)
    test_popularity_labels.append(test_label)

# Pad sequences for test data
# Assuming your model is already trained and loaded
# Evaluate the model on the test data

test_padded = pad_sequences(test_matrixOfVideos, dtype='float32', padding='post', truncating='post')
test_popularity_labels = np.array(test_popularity_labels)
shuffled_test, shuffled_popularity_test = shuffle(test_padded, test_popularity_labels, random_state=42)



test_loss = model.evaluate(shuffled_test, shuffled_popularity_test)
print("Test Loss:", test_loss)

# Make predictions on the test set
test_predictions = model.predict(shuffled_test)

# Evaluate additional metrics if needed
# For example, Mean Absolute Error (MAE) on test data
test_mae = np.mean(np.abs(test_predictions - shuffled_popularity_test))
print("Test Mean Absolute Error:", test_mae)

# Print predictions and original labels for each test video
for i in range(len(test_predictions)):
    print(f"Video {i + 1} - Prediction: {test_predictions[i][0]}, Original Label: {shuffled_popularity_test[i]}")



1/1 [==============================] - 0s 39ms/step - loss: 0.0015
Test Loss: 0.0015440525021404028
1/1 [==============================] - 0s 35ms/step
Test Mean Absolute Error: 0.02140288141088128
Video 1 - Prediction: 0.016829002648591995, Original Label: [0.02942262]
Video 2 - Prediction: -0.11960901319980621, Original Label: [0.02942262]
Video 3 - Prediction: 0.016829002648591995, Original Label: [0.02966094]
Video 4 - Prediction: 0.016829002648591995, Original Label: [0.02982401]
Video 5 - Prediction: 0.016829002648591995, Original Label: [0.02982401]
Video 6 - Prediction: 0.016829002648591995, Original Label: [0.02991047]
Video 7 - Prediction: 0.016829002648591995, Original Label: [0.02991047]
Video 8 - Prediction: 0.016829002648591995, Original Label: [0.02991047]
Video 9 - Prediction: 0.016829002648591995, Original Label: [0.02942262]
Video 10 - Prediction: 0.016829002648591995, Original Label: [0.02982401]
Video 11 - Prediction: 0.016829002648591995, Original Label: [0.0296609